<a href="https://colab.research.google.com/github/theventurecity/analytics/blob/master/Pipeline_Basics_Cohort_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![TheVentureCity](https://theventure.city/wp-content/uploads/2017/06/Theventurecity-logoweb-1.png)

# Pipeline Basics: Cohort Analysis
1. Read raw event log data from a CSV
2. Transform that data into meaningful insights
3. Write the transformed data to Google Sheets 
4. Visualize insights in Google Data Studio

## Before you begin

- This notebook is shared with read-only access. To run this notebook yourself, first click "**Open in Playground**" in the toolbar above. That will create a separate instance that you can run and/or save a copy of to your own Google Drive. 

- To run each cell, hit **Shift-Enter**, which will run the contents of the active cell and move to the next cell. This includes the markup cells (such as this one).

- When you run the first block of Python code, you will get a message that says, "**Warning: This notebook was not authored by Google.**" Please be aware that we are **NOT** accessing your data shared with Google or reading data and credentials from other sessions. This notebook reads data from GitHub and writes to a Google Sheet that only you have access to and can control. We recommend you click the box to "**Reset all runtimes before running**" for extra information security.

## Import relevant Python libraries

In [0]:
### Pandas, Numpy, and date functions to read the data from its source
### and manipulate it in memory
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime
import math


### Google account authentication to give the code access to your Google account
### and gspread to enable writing to Google Sheets
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread
from gspread_dataframe import set_with_dataframe


### The IPython.display library allows us to embed an iFrame within this 
### notebook
from IPython.display import IFrame


### To run this with functions from TheVentureCity's GitHub repository,
### clone the repository to the Google Colaboratory runtime environment
### and then import the code found in growth_accounting.py into a library
### called "ga". This code allows us to run pre-existing functions rather than
### having to spell them out within the notebook
### THIS IS ONLY APPLICABLE IF YOU WANT TO ACCESS THOSE FUNCTIONS ###
### THIS VERSION OF THE NOTEBOOK DEFINES THE FUNCTIONS INLINE     ###
# !if [ ! -d "analytics" ]; then git clone https://github.com/theventurecity/analytics.git; fi
# from importlib.machinery import SourceFileLoader
# ga = SourceFileLoader('growth_accounting', 'analytics/python/growth_accounting.py').load_module()

## 1. Extract: read raw event log data from a CSV

This example uses a data file for a Sample Company from our GitHub repository. In this step we read the data file into memory as a Pandas dataframe we name "t."

In [0]:
# Edit this filename to your local filename.csv if using a local CSV file
filename = 'https://raw.githubusercontent.com/theventurecity/Analytics/master/data/SmileCo_transactions.csv'

t = pd.read_csv(filename)
t.tail(10)

,user_id,activity_date
1209696,438E84E2-CDD3-4311-BC67-8B726149CFCB,2019-02-20 02:31:37.0000000
1209697,8CC36A55-4B70-48D6-A67C-16C290D62988,2019-02-20 02:32:50.0000000
1209698,966294A9-F98E-491F-A5F2-2B07B07B6ED7,2019-02-20 02:33:11.0000000
1209699,8130537F-9317-48E5-BA62-19766B6A5032,2019-02-20 02:33:50.0000000
1209700,3AFC060B-B90A-4E5F-B3DF-0FEACA0B0252,2019-02-20 02:34:37.0000000
1209701,FFA89731-278F-48A0-8433-231E7FD7B2C4,2019-02-20 02:34:59.0000000
1209702,7983860B-8D92-4DC3-ADC8-8AACD3A110B4,2019-02-20 02:35:03.0000000
1209703,ffffffff-d707-9c07-0000-000000000000,2019-02-20 02:35:04.0000000
1209704,16A7BE74-F509-4AB8-B043-11533D8F3B5E,2019-02-20 02:36:33.0000000
1209705,16A7BE74-F509-4AB8-B043-11533D8F3B5E,2019-02-20 02:36:35.0000000


## 2. Transform the raw data into meaningful insights
### 2.1 Create Daily Active Users (DAU) dataframe
The **DAU** dataframe aggregates all activity by user and day. 

In [0]:
# The create_dau_df function takes as inputs a dataframe of transactions and 
# the names of the three key event log columns: User ID, Activity Date, and 
# Income Amount (could be revenue or contribution margin). Next it ensures that
# the Activity Date column is a date and the User ID is a string. Then it groups
# all of the transaction records to calculate the sum of the Income Amount
# by User ID and Activity Date combination

def create_dau_df(transactions, 
                  user_id = 'user_id', 
                  activity_date = 'activity_date', 
                  inc_amt = 'inc_amt'):
    
    # Ensure correct data types
    transactions[activity_date] = pd.to_datetime(transactions[activity_date]).dt.date
    transactions[user_id] = transactions[user_id].astype('str')
    
    
    # If there is no inc_amt available in the data set, add a column of ones
    if inc_amt is None:
        transactions['inc_amt'] = 1
        inc_amt = 'inc_amt'
        
    
    # Group by user_id and activity_date, calculate the sum of the inc_amt
    # and return standardized names for each column
    dau = (transactions
           .groupby([user_id, activity_date], as_index = False)
           .agg({'inc_amt' : 'sum'})
           .rename(columns = {user_id : 'user_id', 
                              activity_date : 'activity_date', 
                              inc_amt : 'inc_amt'})
          )
        
    return dau
  
  
# Run the function above and show the first ten rows  
dau = create_dau_df(t, 
                    user_id = 'user_id', 
                    activity_date = 'activity_date', 
                    inc_amt = None)
dau.head(10)

,user_id,activity_date,inc_amt
0,00000000-0000-d200-0000-000000000000,2018-04-12,1
1,00000000-0000-d200-0000-000000000000,2018-04-14,1
2,00000000-0000-d200-0000-000000000000,2018-04-15,1
3,00000000-0000-d200-0000-000000000000,2018-04-16,1
4,00000000-0006-a79c-0000-000000000000,2019-02-10,2
5,00000000-0006-a79c-0000-000000000000,2019-02-11,1
6,00000000-000a-f125-0000-000000000000,2018-09-11,3
7,00000000-000c-602b-0000-000000000000,2018-09-21,1
8,00000000-000c-602b-0000-000000000000,2018-09-22,1
9,00000000-000c-602b-0000-000000000000,2018-09-23,2


### 2.2 Create a separate dataframe to hold each user's first activity date
Calculate the **first activity date** for each user_id in the DAU dataframe, and store it in its own dataframe (**first_dt**). This step is optional because it can be executed within the **create_dau_decorated_df** function below, but we are including it here to illustrate better the mechanics of what is happening.

In [0]:
# The create_first_dt_df function takes as its input the DAU dataframe created
# above. After creating a copy of the original DAU dataframe so as not to 
# affect the original, it creates a new first_dt dataframe. Using the groupby
# and agg functions, it finds the minimum Activity Date for each User ID. Then 
# it specifies the week ('first_week') and month ('first_month') in which the 
# first Activity Date is found. 

def create_first_dt_df(dau_df):
    print('Creating first_dt dataframe')
    
    # Create copy of input dataframe
    dau = dau_df.copy()
    
    # Use groupby to find the minimum activity_date for each user_id
    first_dt = (dau.groupby(['user_id'], as_index = False)
                .agg({'activity_date' : 'min'})
                .rename(columns = { 'activity_date' : 'first_dt' })
               )
    
    # Ensure that the first_dt field is a date
    first_dt['first_dt'] = pd.to_datetime(first_dt['first_dt']).dt.date
    
    # Add two new columns with the first_week and first_month of the first_dt
    first_dt['first_week'] = pd.to_datetime(first_dt['first_dt']).dt.to_period('W')
    first_dt['first_month'] = pd.to_datetime(first_dt['first_dt']).dt.to_period('M')
    
    return first_dt
  
  
# Run the function above and show the first ten rows  
first_dt = create_first_dt_df(dau)
first_dt.head(10)

Creating first_dt dataframe


,user_id,first_dt,first_week,first_month
0,00000000-0000-d200-0000-000000000000,2018-04-12,2018-04-09/2018-04-15,2018-04
1,00000000-0006-a79c-0000-000000000000,2019-02-10,2019-02-04/2019-02-10,2019-02
2,00000000-000a-f125-0000-000000000000,2018-09-11,2018-09-10/2018-09-16,2018-09
3,00000000-000c-602b-0000-000000000000,2018-09-21,2018-09-17/2018-09-23,2018-09
4,00000000-000c-7de8-0000-000000000000,2018-06-29,2018-06-25/2018-07-01,2018-06
5,00000000-000f-6f9b-0000-000000000000,2018-11-13,2018-11-12/2018-11-18,2018-11
6,00000000-001c-d249-0000-000000000000,2018-10-22,2018-10-22/2018-10-28,2018-10
7,00000000-001f-fedd-0000-000000000000,2018-10-22,2018-10-22/2018-10-28,2018-10
8,00000000-0034-d414-0000-000000000000,2018-10-21,2018-10-15/2018-10-21,2018-10
9,00000000-0039-d2f3-0000-000000000000,2018-04-26,2018-04-23/2018-04-29,2018-04


### 2.3 Join DAU with First Date
Merge the **dau** dataframe with the **first_dt** dataframe and call it "**DAU Decorated**."

In [0]:
# The create_dau_decorated_df takes the two data frames created above, DAU and
# first_dt, and merges them together based on user_id. This results in a DAU
# dataframe "decorated" with information about the user's first activity date,
# first week, and first month, as shown below. Note: it is not necessary to 
# pass in the first_dt dataframe. If none is provided, the function will run
# create_first_dt_df so it has something to merge to the DAU dataframe.

def create_dau_decorated_df(dau_df, first_dt_df = None):
    print('Creating DAU Decorated dataframe')
    
    # If no first_dt_df is provided, create it
    if first_dt_df is None:
        first_dt_df = create_first_dt_df(dau_df)
        
    # Do a left merge of first_dt_df into dau_df on User ID
    dau_decorated_df = dau_df.merge(first_dt_df, how = 'left', on = 'user_id')

    return dau_decorated_df
  
  
# Run the function above and show the first ten rows  
dau_decorated = create_dau_decorated_df(dau, first_dt_df = first_dt)
dau_decorated.tail(10)

Creating DAU Decorated dataframe


,user_id,activity_date,inc_amt,first_dt,first_week,first_month
502520,ffffffff-fff7-aa1b-0000-000000000000,2018-10-29,2,2018-10-23,2018-10-22/2018-10-28,2018-10
502521,ffffffff-fff7-aa1b-0000-000000000000,2018-11-04,2,2018-10-23,2018-10-22/2018-10-28,2018-10
502522,ffffffff-fff7-aa1b-0000-000000000000,2018-11-06,2,2018-10-23,2018-10-22/2018-10-28,2018-10
502523,ffffffff-fff7-aa1b-0000-000000000000,2018-11-08,2,2018-10-23,2018-10-22/2018-10-28,2018-10
502524,ffffffff-fff7-aa1b-0000-000000000000,2018-11-25,3,2018-10-23,2018-10-22/2018-10-28,2018-10
502525,ffffffff-fff7-ce07-0000-000000000000,2018-12-01,1,2018-12-01,2018-11-26/2018-12-02,2018-12
502526,ffffffff-fff8-9004-0000-000000000000,2018-10-23,1,2018-10-23,2018-10-22/2018-10-28,2018-10
502527,ffffffff-fff9-4654-0000-000000000000,2018-10-23,4,2018-10-23,2018-10-22/2018-10-28,2018-10
502528,ffffffff-fff9-4654-0000-000000000000,2018-10-30,1,2018-10-23,2018-10-22/2018-10-28,2018-10
502529,ffffffff-fffa-9b37-0000-000000000000,2018-10-27,2,2018-10-27,2018-10-22/2018-10-28,2018-10


Combining the basic DAU data with the first date, week, and month for each user, **the DAU Decorated dataframe is our basic building block for many different analyses**. It allows us to use user-level data to inspect engagement, retention, and growth accounting.

### 2.4 Calculate cohort analysis metrics
Now that we have the "DAU Decorated" data frame, we can use it to calculate cohort retention and LTV metrics since they are super-important for an early-stage startup. The **create_xau_decorated_df** function below creates a monthly roll-up of the DAU Decorated data we call MAU Decorated.

In [0]:
# For discrete time period calculations, this helper function sets the variable 
# names in the different dataframes 
def get_time_period_dict(time_period):
    
    time_fields_dict = {
                        'week' : {'grouping_col' : 'Week',
                                  'first_period_col' : 'first_week',
                                  'frequency' : 'Weekly',
                                  'unit' : 'Week',
                                  'period_abbr' : 'W',
                                  'python_period' : 'weeks'
                                  },
                        'month' : {'grouping_col' : 'Month_Year',
                                   'first_period_col' : 'first_month',
                                   'frequency' : 'Monthly',
                                   'unit' : 'Month',
                                   'period_abbr' : 'M',
                                   'python_period' : 'months'
                                  }
                        }
                    
    if time_period in time_fields_dict:
        time_fields = time_fields_dict[time_period]
    else:
        time_fields = None
    
    return time_fields
  
  
  
def increment_period(xau_grouping_col, time_period):
    time_fields = get_time_period_dict(time_period)
    period_abbr = time_fields['period_abbr']
    
    if time_period == 'week':
        start_of_next_period = (pd.to_datetime(pd.PeriodIndex(xau_grouping_col)
                                               .start_time 
                                               + timedelta(weeks = 1)))
    elif time_period == 'month':
        start_of_next_period = (pd.to_datetime(pd.PeriodIndex(xau_grouping_col, 
                                                              freq = period_abbr)
                                               .start_time) 
                                + pd.DateOffset(months = 1))
    else:
        start_of_next_period = None
        
    if start_of_next_period is not None:
        next_period = pd.Series(start_of_next_period).dt.to_period(period_abbr)
    else:
        next_period = None
    
    return next_period
  
  

def create_xau_decorated_df(dau_decorated_df, time_period):
    
    time_fields = get_time_period_dict(time_period)
    grouping_col = time_fields['grouping_col']
    frequency = time_fields['frequency']
    first_period_col = time_fields['first_period_col']
    period_abbr = time_fields['period_abbr']
    
    print('Creating ' + frequency + ' Active Users Decorated dataframe')
    
    groupby_cols = [grouping_col, 'user_id', first_period_col]
            
    dau_decorated = dau_decorated_df.copy()
    dau_decorated[grouping_col] = pd.to_datetime(dau_decorated['activity_date']).dt.to_period(period_abbr)
    xau = (dau_decorated.groupby(groupby_cols, as_index = False)['inc_amt'].sum())
    xau['Next_' + grouping_col] = increment_period(xau[grouping_col], time_period)
    
    output_cols = [grouping_col, 'user_id', 'inc_amt', first_period_col, 'Next_' + grouping_col]
    xau = xau[output_cols]
    
    return xau

In [0]:
mau_decorated = create_xau_decorated_df(dau_decorated, 'month')
mau_decorated.tail(10)

Creating Monthly Active Users Decorated dataframe


,Month_Year,user_id,inc_amt,first_month,Next_Month_Year
56827,2019-02,ffffffff-fe29-7514-0000-000000000000,1,2019-01,2019-03
56828,2019-02,ffffffff-fe56-e310-0000-000000000000,6,2019-02,2019-03
56829,2019-02,ffffffff-fe8d-63e7-0000-000000000000,2,2019-02,2019-03
56830,2019-02,ffffffff-fec5-8375-0000-000000000000,1,2018-10,2019-03
56831,2019-02,ffffffff-fed5-7da3-0000-000000000000,1,2018-10,2019-03
56832,2019-02,ffffffff-fef6-d125-0000-000000000000,1,2018-10,2019-03
56833,2019-02,ffffffff-ff40-bd27-0000-000000000000,6,2018-10,2019-03
56834,2019-02,ffffffff-ff54-afcb-0000-000000000000,5,2018-10,2019-03
56835,2019-02,ffffffff-ff5b-02fb-0000-000000000000,1,2019-01,2019-03
56836,2019-02,ffffffff-ff61-98d3-0000-000000000000,1,2019-02,2019-03


The **xau_retention_by_cohort_df** function below helps us calculate what happens to the cohort of users that comes in as a new customer each month. It looks at how many of them continue to use the product (expressed as a number and as a percentage), how much income they generate each month, the income per customer, and the cumulative income per customer. This information leads to several insightful visuals as we will see below.

In [0]:
### Calculate the user retention by cohort defined by any weekly or monthly time period
def xau_retention_by_cohort_df(xau_decorated_df, 
                               time_period, 
                               recent_periods_back_to_exclude = 1, 
                               date_limit = None):
    
    time_fields = get_time_period_dict(time_period)
    grouping_col = time_fields['grouping_col']
    first_period_col = time_fields['first_period_col']
    unit = time_fields['unit']
    period_abbr = time_fields['period_abbr']
        
    since_col = '%ss Since First' % unit
    
    if date_limit is not None:
        xau_d = (xau_decorated_df[pd.PeriodIndex(xau_decorated_df[grouping_col], 
                                                 freq = period_abbr)
                                  .start_time <= date_limit].copy())
    else:
        xau_d = xau_decorated_df.copy()
    
    xau_d[since_col] = xau_d[grouping_col] - xau_d[first_period_col]
    
    first_groupby_cols = [first_period_col, grouping_col, since_col]
    
    xau_d = (xau_d.groupby(first_groupby_cols)
             .agg({'inc_amt' : 'sum', 'user_id' : 'nunique'})
             .rename(columns = { 'user_id' : 'cust_ct' })
            )
                    
    second_groupby_cols = [first_period_col]
    
    xau_d['cohort_cust_ct'] = (xau_d.groupby(second_groupby_cols)['cust_ct']
                               .transform('first'))
    xau_d['cum_inc_amt'] = (xau_d.groupby(second_groupby_cols)['inc_amt']
                            .cumsum())
    
    xau_d['cum_inc_per_cohort_cust'] = xau_d['cum_inc_amt'] / xau_d['cohort_cust_ct']
    xau_d['cust_ret_pct'] = xau_d['cust_ct'] / xau_d['cohort_cust_ct']
    
    xau_d = xau_d.reset_index()
    
    if time_period == 'month':
        td = pd.DateOffset(months = recent_periods_back_to_exclude)
    elif time_period == 'week':
        td = timedelta(weeks = recent_periods_back_to_exclude)
    
    last_period = pd.to_datetime(datetime.today() - td).to_period(period_abbr)
    xau_d = xau_d.loc[xau_d[grouping_col] <= last_period]
    
    xau_d[first_period_col] = (pd.PeriodIndex(xau_d[first_period_col], 
                                              freq = period_abbr).start_time )
    xau_d[grouping_col] = (pd.PeriodIndex(xau_d[grouping_col], 
                                          freq = period_abbr).start_time )
    
    
       
    return xau_d


In [0]:
mau_retention_by_cohort = xau_retention_by_cohort_df(mau_decorated, 'month')
mau_retention_by_cohort.tail(10)

,first_month,Month_Year,Months Since First,inc_amt,cust_ct,cohort_cust_ct,cum_inc_amt,cum_inc_per_cohort_cust,cust_ret_pct
95,2018-11-01,2018-11-01,0,11029,985,985,11029,11.196954,1.000000
96,2018-11-01,2018-12-01,1,4968,251,985,15997,16.240609,0.254822
97,2018-11-01,2019-01-01,2,2301,139,985,18298,18.576650,0.141117
98,2018-11-01,2019-02-01,3,1036,85,985,19334,19.628426,0.086294
99,2018-12-01,2018-12-01,0,10374,672,672,10374,15.437500,1.000000
100,2018-12-01,2019-01-01,1,2751,166,672,13125,19.531250,0.247024
101,2018-12-01,2019-02-01,2,1039,86,672,14164,21.077381,0.127976
102,2019-01-01,2019-01-01,0,11646,881,881,11646,13.219069,1.000000
103,2019-01-01,2019-02-01,1,4128,247,881,15774,17.904654,0.280363
104,2019-02-01,2019-02-01,0,2565,437,437,2565,5.869565,1.000000


## 3. Load: write the transformed data to Google Sheets 
### Establish connection to Google Sheets for writing output files
The first time you run this cell, or after some time of inactivity, you will be asked to click on a link. That link will take you to a new tab that will authorize this script to write to Google Sheets spreadsheets in your Google Account. To enable this feature, copy the code you get into the box below and hit Enter.

In [0]:
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

### Function to facilitate writing from Pandas to Google Sheets
For use further down in this notebook.

In [0]:
def write_to_google_sheet(dataframe, spreadsheet_key, worksheet_name, goog_creds = gc):
  
  sh = goog_creds.open_by_key(spreadsheet_key)
  
  ws = None
  worksheet_list = sh.worksheets()
  for worksheet in worksheet_list:
    if worksheet.title == worksheet_name:
      ws = worksheet
  if ws is None:
    ws = sh.add_worksheet(title = worksheet_name, rows="1", cols = "1")
    
  set_with_dataframe(ws, dataframe, row=1, col=1, include_index=False, 
                     include_column_header=True, resize=True, allow_formulas=True)

### Create Google Sheet for writing output files if one is not already established

***Warning: you must choose from Options A and B below***.

**Option A**: If you already have a Google Sheet where you store transformed data for analytics, go to that sheet, copy the long ID string from the sheet's URL in a browser, and paste it between the quotes below. Then uncomment the code and run the cell. Do NOT run the code for Option B without commenting it out.

This option is especially important if you have previously linked a reporting dashboard to this Google Sheet and would like to use this ETL process to update that dashboard.

In [0]:
### Option A Code
GOOGLE_SHEET_KEY = '1-XnO_eWkRwX-E1fiA2Jkbe3kJvoyoPFsdeW7vnF6zS0' 

**Option B**: If you are running this for the first time and do not already have a Google Sheet where you store transformed data for analytics, uncomment and run the cell below to set the GOOGLE_SHEET_KEY variable, which will be used later in the process. Be sure to replace the "Sample Company Analytics" filename with one of your own.

In [0]:
### Option B Code
# GOOGLE_SHEET_KEY = gc.create('Sample Company Analytics').id

After executing the cell above, a new spreadsheet with the name you supply will be shown in your sheets list on [sheets.google.com](http://sheets.google.com/). To go directly to the newly created Google Sheet, run the code block below and visit the link that it outputs. You can keep this Sheet open in a separate tab to see the data get updated whenever **write_to_google_sheet** (defined above) is called.

In [0]:
print('https://docs.google.com/spreadsheets/d/' + GOOGLE_SHEET_KEY)

https://docs.google.com/spreadsheets/d/1-XnO_eWkRwX-E1fiA2Jkbe3kJvoyoPFsdeW7vnF6zS0


In [0]:
write_to_google_sheet(mau_retention_by_cohort, GOOGLE_SHEET_KEY, 'MAU Retention by Cohort')

## 4. Visualize insights in Google Data Studio
A Google DataStudio dashboard preconfigured to read from the Google Sheet created above to visualize the data is [available at this link](https://datastudio.google.com/open/1xjS__Q6ZUXuUUARkgRvY4spYUw1ePksV) or by clicking on the Google DataStudio logo at the bottom of the chart embedded below. It is available in read-only mode for you to copy, link to your own Google Sheet tabs, and see your own data visualized.

In [0]:
IFrame('https://datastudio.google.com/embed/reporting/1xjS__Q6ZUXuUUARkgRvY4spYUw1ePksV/page/EOyj', 
       width=600, 
       height=450)